In [8]:
import numpy as np
import os
import random
from random import shuffle
import scipy.misc
import scipy
import time
import skimage.io
from skimage.io import imread
import keras
import keras.callbacks
from keras import optimizers, metrics, regularizers
from keras import backend as K
from keras.models import Sequential, Model, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization, SeparableConv2D, core
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, Cropping2D, concatenate, Input
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.activations import softmax
import keras.losses
import matplotlib
import sklearn.metrics
from sklearn.metrics import classification_report, confusion_matrix

In [9]:
class_weight_one = 1
class_weight_two = 1
batch_size = 5
epochs_2_train = 10
validPercent = 0.1
image_folder = '/data/Ro_ImageData/Low SN ratio ceramic images/' #Source of images
highSN_label = '/data/Ro_ImageData/32xOTSU/' #Otsu images
lowSN_label = '/data/Ro_ImageData/32xOTSULow'
highSN_folder = '/data/Ro_ImageData/High SN ratio ceramic images/'
lowSN_folder = '/data/Ro_ImageData/Low SN ratio ceramic images/'
prediction_folder = '/home/dspuser/MA499/Predictions/' #Folder for predictions

In [10]:
def get_sample(sample, highSN_folder, lowSN_folder, highSN_label, lowSN_label):
    if ('_C' in sample):
    #If it's a low SN image
        inputImgPath = os.path.join(lowSN_folder, sample)
        maskImgPath = os.path.join(lowSN_label, sample)
    else:
    #If it's a high SN image
        inputImgPath = os.path.join(highSN_folder, sample)
        maskImgPath = os.path.join(highSN_label, sample)
    img_int = np.array(imread(inputImgPath), dtype = 'float32')/255
    lbl_int = np.array(imread(maskImgPath), dtype = 'float32')/2**16
    lbl_int[:,:,1] = 1-lbl_int[:,:,0]
    lbl_int = lbl_int[:,:,0:2]
    return img_int,lbl_int

def get_sample_test(sample,test_folder):
    img_int = np.array(imread(test_folder + sample), dtype = 'float32')/255
    return img_int

def data_generator(file_name_list,batch_size): #randomly sampled instances from file name list with batch size
    while True:
        batch_filenames = np.random.choice(file_name_list,batch_size)
        batch_input = []
        batch_output = []
        for i_filename in batch_filenames:
            Ai_img, Ai_mask = get_sample(i_filename,highSN_folder,lowSN_folder,highSN_label,lowSN_label)
            batch_input += [Ai_img]
            batch_output += [Ai_mask]
            #print(i_filename)
        batch_img = np.expand_dims(np.array(batch_input,dtype='float32'),axis=-1)
        batch_mask = np.array(batch_output,dtype='float32')
        yield (batch_img,batch_mask)

def Weighted_Binary_CrossEntropy(y_true_n,y_pred_n):
    b_ce = K.binary_crossentropy(y_true_n,y_pred_n)
    y_true = K.cast(K.expand_dims(K.argmax(y_true_n,axis=-1),axis=-1),dtype='float32')
    y_pred = K.cast(K.expand_dims(K.argmax(y_pred_n,axis=-1),axis=-1),dtype='float32')
    #Pixel Disparity
    one_weight = class_weight_one
    zero_weight = class_weight_two
    weight_vector = y_true * one_weight + (1.-y_true)*zero_weight
    weighted_b_ce = weight_vector*b_ce
    return K.mean(weighted_b_ce)

keras.losses.Weighted_Binary_CrossEntropy = Weighted_Binary_CrossEntropy

def makeSampleList(highSN_folder,lowSN_folder,sampleSize,prop):
    sampleList = []
    highSN = os.listdir(highSN_folder)
    #highSN = [os.path.join(highSN_folder,file) for file in highSN]
    lowSN = os.listdir(lowSN_folder)
    #lowSN = [os.path.join(lowSN_folder,file) for file in lowSN]
    highSample = list(np.random.choice(highSN,sampleSize-round(sampleSize*prop)))
    lowSample = list(np.random.choice(lowSN,round(sampleSize*prop)))
    sampleListAll = highSample + lowSample
    for file in sampleListAll:
        if os.path.splitext(file)[1] == '.tiff':
            sampleList.append(file)
    return sampleList

In [35]:
sampleList = makeSampleList(highSN_folder,lowSN_folder,2000,0.25)

indexes_valid = random.sample(range(0,len(sampleList)),int(validPercent*float(len(sampleList))))
indexes_train = [x for x in range(0,len(sampleList)) if x not in indexes_valid]
training_list = [x for ind,x in enumerate(sampleList) if ind in indexes_train]
valid_list = [x for ind,x in enumerate(sampleList) if ind in indexes_valid]
num_train_calls = int(float(len(training_list)+1)/float(batch_size))
num_valid_calls = int(float(len(valid_list)+1)/float(batch_size))

In [ ]:
#### Network is built ####
network = Sequential()
network.add(Conv2D(10,(3,3),activation = 'relu',kernel_regularizer = regularizers.l2(.01),input_shape = (1024,1024,1),
padding = 'same'))
network.add(keras.layers.BatchNormalization())
network.add(Conv2D(10,(3,3),activation = 'relu',kernel_regularizer = regularizers.l2(.01),padding='same'))
network.add(keras.layers.BatchNormalization())
network.add(Dropout(0.25))
network.add(Conv2D(10,(3,3),activation = 'relu',kernel_regularizer = regularizers.l2(.01),padding='same'))
network.add(keras.layers.BatchNormalization())
network.add(Dropout(0.25))
network.add(Conv2D(10,(3,3),activation = 'relu',kernel_regularizer = regularizers.l2(.01),padding='same'))
network.add(keras.layers.BatchNormalization())
network.add(Dropout(0.25))
network.add(Conv2D(2,(1,1),activation = 'softmax',kernel_regularizer = regularizers.l2(.01),padding='same'))
sgd = optimizers.SGD(lr = 0.0001, decay = 1e-8, momentum = 0.9, nesterov = False)
network.compile(loss = Weighted_Binary_CrossEntropy, optimizer = sgd, metrics = ['accuracy'])
network.summary()
network_training = network.fit_generator(data_generator(training_list,batch_size),steps_per_epoch = 50, 
                                         epochs = epochs_2_train, verbose = 1,validation_data = data_generator(valid_list,batch_size),
                                         validation_steps = num_valid_calls)
TP = 0 #True Positives
TN = 0 #True Negatives
FP = 0 #False Positives
FN = 0 #False Negatives

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_76 (Conv2D)           (None, 1024, 1024, 10)    100       
_________________________________________________________________
batch_normalization_61 (Batc (None, 1024, 1024, 10)    40        
_________________________________________________________________
conv2d_77 (Conv2D)           (None, 1024, 1024, 10)    910       
_________________________________________________________________
batch_normalization_62 (Batc (None, 1024, 1024, 10)    40        
_________________________________________________________________
dropout_46 (Dropout)         (None, 1024, 1024, 10)    0         
_________________________________________________________________
conv2d_78 (Conv2D)           (None, 1024, 1024, 10)    910       
_________________________________________________________________
batch_normalization_63 (Batc (None, 1024, 1024, 10)  

/opt/conda/lib/python3.7/site-packages/skimage/external/tifffile/tifffile.py:2616: RuntimeWarning: py_decodelzw encountered unexpected end of stream
  strip = decompress(strip)
/opt/conda/lib/python3.7/site-packages/skimage/external/tifffile/tifffile.py:2551: UserWarning: unpack: buffer size must be a multiple of element size
  warnings.warn("unpack: %s" % e)


50/50 [==============================] - 42s 839ms/step - loss: 1.0601 - accuracy: 0.7202 - val_loss: 1.2053 - val_accuracy: 0.3227
Epoch 2/10
49/50 [============================>.] - ETA: 0s - loss: 0.8543 - accuracy: 0.8156

In [34]:
for filename in valid_list: #Use for new lists
    img,lbl = get_sample(filename,highSN_folder,lowSN_folder,highSN_label,lowSN_label)
    lbl = np.argmax(lbl,axis=-1)
    img = np.expand_dims(img,axis=0)
    img = np.expand_dims(img,axis=-1)
    img_pred = network.predict(img)
    pred = np.argmax(img_pred[0,:,:,:],axis=-1)
    TP += np.sum(pred*lbl,dtype='float32')
    TN += np.sum((1-pred)*(1-lbl),dtype='float32')
    FP += np.sum((1-pred)*lbl,dtype='float32')
    FN += np.sum(pred*(1-lbl),dtype='float32')

ACC = (TP + TN)/(TP + TN + FP + FN) #Accuracy
Recall = (TP)/(TP + FN)
Precision = (TP)/(TP + FP)
MCC = ((TP * TN) - (FP * FN))/np.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))

np.savetxt('ValidationMetrics2.txt',(ACC,Recall,Precision,MCC))
np.savetxt('ValidationLoss2.txt',network_training.history['val_loss'])
np.savetxt('ValidationAcc2.txt',network_training.history['val_accuracy'])
test_list = valid_list
network.save("H85_L15.h5") 
model = keras.models.load_model("H85_L15.h5")

In [25]:
### Generate predictions from trained model ###
for filename in test_list:
    img,lbl = get_sample(filename, highSN_folder, lowSN_folder, highSN_label, lowSN_label)
    lbl = np.argmax(lbl,axis=-1)
    img = np.expand_dims(img,axis=0)
    img = np.expand_dims(img,axis=-1)
    img_pred = model.predict(img)
    pred = np.argmax(img_pred[0,:,:,:],axis=-1)
    matplotlib.image.imsave(prediction_folder+filename,pred)

FileNotFoundError: [Errno 2] No such file or directory: '/home/dspuser/MA499/Predictions2/Tile_r009_c028.tiff'